# Titanic - ML from disaster

Use machine learning to create a model that predicts which passengers survived the Titanic shipwreck.

In [1]:
import numpy as np
import pandas as pd

np.random.seed(13)

In [2]:
# Import training and test sets
train_set = pd.read_csv("train.csv")
test_set = pd.read_csv("test.csv")

# Review dataset
train_set.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [7]:
test_set.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,0,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,1,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,0,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,0,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,1,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [8]:
# Drop columns deemed arbitrary
train = train_set.drop(['Name', 'Ticket', 'Cabin'], axis=1)
test = test_set.drop(['Name', 'Ticket', 'Cabin'], axis=1)

In [4]:
# Review shape of training data
train.shape

(891, 9)

In [5]:
# Review shape of test data
test.shape

(418, 9)

In [9]:
# Set target variable and remove target from input data.

train_y = train[['Survived']]
test_y = test[['Survived']]

train_inputs = train.drop(['Survived'], axis=1)
test_inputs = test.drop(['Survived'], axis=1)

In [11]:
test_inputs.head()

,PassengerId,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,892,3,male,34.5,0,0,7.8292,Q
1,893,3,female,47.0,1,0,7.0000,S
2,894,2,male,62.0,0,0,9.6875,Q
3,895,3,male,27.0,0,0,8.6625,S
4,896,3,female,22.0,1,1,12.2875,S


In [12]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder

from sklearn.preprocessing import FunctionTransformer

In [ ]:
# Reveiw training dataset
train_inputs.head()

In [13]:
# Review column types
train_inputs.dtypes

PassengerId      int64
Pclass           int64
Sex             object
Age            float64
SibSp            int64
Parch            int64
Fare           float64
Embarked        object
dtype: object

In [38]:
train_inputs['Pclass']=train_inputs['Pclass'].astype('object')

In [39]:
train_inputs.dtypes

PassengerId      int64
Pclass          object
Sex             object
Age            float64
SibSp            int64
Parch            int64
Fare           float64
Embarked        object
dtype: object

In [40]:
# Identify the numerical columns
numeric_columns = train_inputs.select_dtypes(include=[np.number]).columns.to_list()

# Identify the categorical columns
categorical_columns = train_inputs.select_dtypes('object').columns.to_list()

In [41]:
# Confirm numeric cols
numeric_columns

['PassengerId', 'Age', 'SibSp', 'Parch', 'Fare']

In [42]:
# Confirm cat cols
categorical_columns

['Pclass', 'Sex', 'Embarked']

In [43]:
# Create numeric pipeline
numeric_transformer = Pipeline(steps=[
                ('imputer', SimpleImputer(strategy='mean')),
                ('scaler', StandardScaler())])

In [44]:
# Create cat pipeline
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value='unknown')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))])

In [45]:
# Create preprocessor
preprocessor = ColumnTransformer([
        ('num', numeric_transformer, numeric_columns),
        ('cat', categorical_transformer, categorical_columns)],
        remainder='drop')

In [46]:
# #Fit and transform the train data
train_x = preprocessor.fit_transform(train_inputs)
train_x.shape

(891, 14)

In [48]:
#Fit and transform the test data
test_x = preprocessor.transform(test_inputs)

test_x.shape

(418, 14)

In [49]:
train_y

,Survived
0,0
1,1
2,1
3,1
4,0
...,...
886,0
887,1
888,0
889,1


In [50]:
# Determine a baseline accuracy using most frequent stra

from sklearn.dummy import DummyClassifier
from sklearn.metrics import accuracy_score

dummy_clf = DummyClassifier(strategy="most_frequent")

dummy_clf.fit(train_x, train_y)

DummyClassifier(strategy='most_frequent')

In [51]:
# Baseline Train Accuracy
dummy_train_pred = dummy_clf.predict(train_x)

baseline_train_acc = accuracy_score(train_y, dummy_train_pred)

print('Baseline Train Accuracy: {}' .format(baseline_train_acc))

Baseline Train Accuracy: 0.6161616161616161


In [52]:
# Baseline Test Accuracy
dummy_test_pred = dummy_clf.predict(test_x)

baseline_test_acc = accuracy_score(test_y, dummy_test_pred)

print('Baseline Test Accuracy: {}' .format(baseline_test_acc))

Baseline Test Accuracy: 0.6363636363636364


In [53]:
# Create a single layer neural network model

import tensorflow as tf
from tensorflow import keras

np.random.seed(13)
tf.random.set_seed(13)

NN = keras.models.Sequential()

NN.add(keras.layers.Input(shape=train_x.shape[1]))
NN.add(keras.layers.Dense(16, activation='relu'))
NN.add(keras.layers.Dense(1, activation='sigmoid'))

In [54]:
# Compile model

#Optimizer:
adam = keras.optimizers.Adam(learning_rate=0.01)

NN.compile(loss='binary_crossentropy', optimizer=adam, metrics=['accuracy'])

In [55]:
# Fit the model
history = NN.fit(train_x, train_y, 
                    validation_data=(test_x, test_y), 
                    epochs=20, batch_size=100)

Epoch 1/20


9/9 [==============================] - 1s 40ms/step - loss: 0.6369 - accuracy: 0.6498 - val_loss: 0.5225 - val_accuracy: 0.8278
Epoch 2/20
9/9 [==============================] - 0s 8ms/step - loss: 0.5195 - accuracy: 0.7520 - val_loss: 0.4157 - val_accuracy: 0.8158
Epoch 3/20
9/9 [==============================] - 0s 9ms/step - loss: 0.4705 - accuracy: 0.7789 - val_loss: 0.3555 - val_accuracy: 0.8254
Epoch 4/20
9/9 [==============================] - 0s 9ms/step - loss: 0.4406 - accuracy: 0.8025 - val_loss: 0.3244 - val_accuracy: 0.8828
Epoch 5/20
9/9 [==============================] - 0s 8ms/step - loss: 0.4284 - accuracy: 0.8058 - val_loss: 0.3120 - val_accuracy: 0.8995
Epoch 6/20
9/9 [==============================] - 0s 9ms/step - loss: 0.4220 - accuracy: 0.8114 - val_loss: 0.2956 - val_accuracy: 0.9115
Epoch 7/20
9/9 [==============================] - 0s 10ms/step - loss: 0.4160 - accuracy: 0.8114 - val_loss: 0.2855 - val_accuracy: 0.9043
Epoch 8/20
9/9 [==============

In [56]:
train_scores = NN.evaluate(train_x, train_y, verbose=0)

print(f"Train {NN.metrics_names[0]}: {train_scores[0]:.2f}")

print(f"Train {NN.metrics_names[1]}: {train_scores[1]*100:.2f}%")

Train loss: 0.37
Train accuracy: 84.40%


In [57]:
test_scores = NN.evaluate(test_x, test_y, verbose=0)

print(f"Test {NN.metrics_names[0]}: {test_scores[0]:.2f}")

print(f"Test {NN.metrics_names[1]}: {test_scores[1]*100:.2f}%")

Test loss: 0.36
Test accuracy: 77.99%


In [58]:
# Create a multi-layer Deep Neural Network model

np.random.seed(13)
tf.random.set_seed(13)

DNN = keras.models.Sequential()

DNN.add(keras.layers.Input(shape=train_x.shape[1]))
DNN.add(keras.layers.Dense(16, activation='relu'))
DNN.add(keras.layers.Dense(12, activation='relu'))
DNN.add(keras.layers.Dense(8, activation='relu'))
DNN.add(keras.layers.Dense(1, activation='sigmoid'))

In [59]:
# Compile model
adam = keras.optimizers.Adam(learning_rate=0.001)
DNN.compile(loss='binary_crossentropy', optimizer=adam, metrics=['accuracy'])

In [60]:
# Apply early stopping to prevent overfitting

from tensorflow.keras.callbacks import EarlyStopping

earlystop = EarlyStopping(patience=3, verbose=1)

callback = [earlystop]

DNN.fit(train_x, train_y, validation_data=(test_x, test_y), 
            epochs=100, batch_size=100, callbacks=callback)

Epoch 1/100
9/9 [==============================] - 1s 33ms/step - loss: 0.7097 - accuracy: 0.4366 - val_loss: 0.6976 - val_accuracy: 0.5024
Epoch 2/100
9/9 [==============================] - 0s 9ms/step - loss: 0.6906 - accuracy: 0.5320 - val_loss: 0.6826 - val_accuracy: 0.5478
Epoch 3/100
9/9 [==============================] - 0s 9ms/step - loss: 0.6760 - accuracy: 0.5836 - val_loss: 0.6698 - val_accuracy: 0.6316
Epoch 4/100
9/9 [==============================] - 0s 10ms/step - loss: 0.6620 - accuracy: 0.6150 - val_loss: 0.6570 - val_accuracy: 0.6627
Epoch 5/100
9/9 [==============================] - 0s 10ms/step - loss: 0.6497 - accuracy: 0.6139 - val_loss: 0.6435 - val_accuracy: 0.6555
Epoch 6/100
9/9 [==============================] - 0s 11ms/step - loss: 0.6364 - accuracy: 0.6105 - val_loss: 0.6305 - val_accuracy: 0.6483
Epoch 7/100
9/9 [==============================] - 0s 9ms/step - loss: 0.6231 - accuracy: 0.6150 - val_loss: 0.6174 - val_accuracy: 0.6483
Epoch 8/100
9/9 [======

In [61]:
train_scores = DNN.evaluate(train_x, train_y, verbose=0)

print(f"Train {DNN.metrics_names[0]}: {train_scores[0]:.2f}")

print(f"Train {DNN.metrics_names[1]}: {train_scores[1]*100:.2f}%")

Train loss: 0.40
Train accuracy: 83.05%


In [62]:
test_scores = DNN.evaluate(test_x, test_y, verbose=0)

print(f"Test {DNN.metrics_names[0]}: {test_scores[0]:.2f}")

print(f"Test {DNN.metrics_names[1]}: {test_scores[1]*100:.2f}%")

Test loss: 0.34
Test accuracy: 79.90%


In [63]:
# Now let's try a RandomForestClassifier model

from sklearn.ensemble import RandomForestClassifier 

rnd_clf = RandomForestClassifier(n_estimators=100, max_depth=10, min_samples_split=2, n_jobs=-1) 

rnd_clf.fit(train_x, train_y)

C:\Users\chadu\anaconda3\lib\site-packages\sklearn\base.py:1152: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


RandomForestClassifier(max_depth=10, n_jobs=-1)

In [64]:
train_y_pred = rnd_clf.predict(train_x)

train_acc = accuracy_score(train_y, train_y_pred)

print('Train acc: {}' .format(train_acc))

Train acc: 0.9517396184062851


In [65]:
test_y_pred = rnd_clf.predict(test_x)

test_acc = accuracy_score(test_y, test_y_pred)

print('Test acc: {}' .format(test_acc))

Test acc: 0.8181818181818182


# SVC Model 

In [66]:
from sklearn.svm import SVC

LSVC = SVC(kernel="linear")

LSVC.fit (train_x, train_y)

C:\Users\chadu\anaconda3\lib\site-packages\sklearn\utils\validation.py:1183: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


SVC(kernel='linear')

In [67]:
train_y_pred = LSVC.predict(train_x)

train_acc = accuracy_score(train_y, train_y_pred)

print('Train acc: {}' .format(train_acc))

Train acc: 0.7867564534231201


In [68]:
test_y_pred = LSVC.predict(test_x)

test_acc = accuracy_score(test_y, test_y_pred)

print('Test acc: {}' .format(test_acc))

Test acc: 1.0


## SVC Poly

In [69]:
from sklearn.svm import SVC
PSVC = SVC(kernel="poly", degree=3, coef0=1, C=.1)
PSVC.fit (train_x, train_y)

C:\Users\chadu\anaconda3\lib\site-packages\sklearn\utils\validation.py:1183: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


SVC(C=0.1, coef0=1, kernel='poly')

In [70]:
train_y_pred = PSVC.predict(train_x)

train_acc = accuracy_score(train_y, train_y_pred)

print('Train acc: {}' .format(train_acc))

Train acc: 0.8338945005611672


In [71]:
test_y_pred = PSVC.predict(test_x)

test_acc = accuracy_score(test_y, test_y_pred)

print('Test acc: {}' .format(test_acc))

Test acc: 0.9425837320574163


## SVC RBF

In [72]:
rbf_svm = SVC(kernel="rbf", C=2, gamma='scale')

rbf_svm.fit(train_x, train_y)

C:\Users\chadu\anaconda3\lib\site-packages\sklearn\utils\validation.py:1183: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


SVC(C=2)

In [73]:
train_y_pred = rbf_svm.predict(train_x)
train_acc = accuracy_score(train_y, train_y_pred)
print('Train acc: {}' .format(train_acc))

Train acc: 0.8462401795735129


In [74]:
test_y_pred = rbf_svm.predict(test_x)
test_acc = accuracy_score(test_y, test_y_pred)
print('Test acc: {}' .format(test_acc))

Test acc: 0.9138755980861244
